## Diego Emilio Muñiz Ramirez

# 621784, 04/02/26

## Introducción
En problemas reales de ciencia de datos, los conjuntos de datos rara vez se encuentran listos para su uso directo en modelos predictivos. Es común encontrar variables categóricas, escalas inconsistentes y relaciones redundantes entre variables.

El objetivo de este reporte es construir un modelo de regresión lineal múltiple que permita predecir la calificación final (G3) de estudiantes, abordando explícitamente la preparación de datos, la selección adecuada de características y la evaluación del desempeño del modelo.

##Exploración del conjunto de datos
Se realiza una exploración inicial para comprender la estructura del conjunto de datos, el tipo de variables disponibles y su posible relación con la calificación final.

In [1]:
import pandas as pd

df = pd.read_csv("A1.3 Calificaciones.csv")
df.head()
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 395 entries, 0 to 394
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Escuela         395 non-null    object
 1   Sexo            395 non-null    object
 2   Edad            395 non-null    int64 
 3   HorasDeEstudio  395 non-null    int64 
 4   Reprobadas      395 non-null    int64 
 5   Internet        395 non-null    object
 6   Faltas          395 non-null    int64 
 7   G1              395 non-null    int64 
 8   G2              395 non-null    int64 
 9   G3              395 non-null    int64 
dtypes: int64(7), object(3)
memory usage: 31.0+ KB


,Edad,HorasDeEstudio,Reprobadas,Faltas,G1,G2,G3
count,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000
mean,16.696203,2.035443,0.334177,5.708861,10.908861,10.713924,10.415190
std,1.276043,0.839240,0.743651,8.003096,3.319195,3.761505,4.581443
min,15.000000,1.000000,0.000000,0.000000,3.000000,0.000000,0.000000
25%,16.000000,1.000000,0.000000,0.000000,8.000000,9.000000,8.000000
50%,17.000000,2.000000,0.000000,4.000000,11.000000,11.000000,11.000000
75%,18.000000,2.000000,0.000000,8.000000,13.000000,13.000000,14.000000
max,22.000000,4.000000,3.000000,75.000000,19.000000,19.000000,20.000000


## Interpretación
El conjunto de datos contiene 395 observaciones y 10 variables, sin valores faltantes.
Se identifican variables cuantitativas (edad, horas de estudio, faltas, G1, G2) y categóricas (escuela, sexo, acceso a internet), lo cual implica la necesidad de transformaciones previas al modelado.

## Preparación y limpieza de los datos
Para que los datos sean adecuados para un modelo de regresión lineal múltiple, se realizan transformaciones sobre las variables categóricas y numéricas.

In [2]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

X = df.drop(columns=["G3"])
y = df["G3"]

categorical = ["Escuela", "Sexo", "Internet"]
numerical = [c for c in X.columns if c not in categorical]

preprocess = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(drop="first"), categorical),
        ("num", StandardScaler(), numerical)
    ]
)

## Interpretación
Las variables categóricas se codifican mediante One-Hot Encoding para evitar relaciones ordinales artificiales. Las variables numéricas se estandarizan para prevenir que diferencias de escala afecten el ajuste del modelo.

## Análisis de relaciones entre variables
Se analiza la relación entre las variables cuantitativas y la calificación final
para identificar posibles redundancias o colinealidad.

Para apoyar el proceso de selección de características, se analiza la correlación
entre las variables cuantitativas y la calificación final.


In [3]:
df.corr(numeric_only=True)["G3"].sort_values(ascending=False)

G3                1.000000
G2                0.904868
G1                0.801468
HorasDeEstudio    0.097820
Faltas            0.034247
Edad             -0.161579
Reprobadas       -0.360415
Name: G3, dtype: float64

## Interpretación
Se observa una alta correlación positiva entre las calificaciones parciales G1 y G2 con la calificación final G3, lo cual es consistente con la naturaleza del problema. Esto justifica su inclusión como variables explicativas clave en el modelo.

## Selección de características
La selección de características se realizó considerando:
- Relevancia predictiva respecto a G3
- Reducción de información redundante
- Interpretabilidad del modelo

Se conservaron las calificaciones parciales, variables académicas y variables categóricas para evaluar su impacto conjunto.

## Entrenamiento del modelo de regresión lineal múltiple
Se entrena un modelo de regresión lineal múltiple utilizando un pipeline que integra el preprocesamiento y el entrenamiento, evitando fuga de datos.

In [4]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

model = Pipeline(steps=[
    ("preprocess", preprocess),
    ("regressor", LinearRegression())
])

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

model.fit(X_train, y_train)

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(drop='first'),
                                                  ['Escuela', 'Sexo',
                                                   'Internet']),
                                                 ('num', StandardScaler(),
                                                  ['Edad', 'HorasDeEstudio',
                                                   'Reprobadas', 'Faltas', 'G1',
                                                   'G2'])])),
                ('regressor', LinearRegression())])

## Interpretación
El uso de un pipeline permite que el preprocesamiento (codificación y escalado)
se ajuste únicamente con los datos de entrenamiento y posteriormente se aplique
a los datos de prueba, evitando explícitamente la fuga de datos durante la
evaluación del modelo.


## Evaluación del modelo
El desempeño del modelo se evalúa utilizando métricas tanto en el conjunto de entrenamiento como en el de prueba.

In [5]:
from sklearn.metrics import r2_score, root_mean_squared_error

y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)
rmse_test = root_mean_squared_error(y_test, y_test_pred)

r2_train, r2_test, rmse_test

/Users/diegomuniz/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/Users/diegomuniz/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: overflow encountered in matmul
  return X @ coef_ + self.intercept_
/Users/diegomuniz/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: invalid value encountered in matmul
  return X @ coef_ + self.intercept_


(0.8413834853903517, 0.7809784513196283, 2.119209099738873)

## Interpretación de resultados
- R² entrenamiento: 0.84
- R² prueba: 0.78
- RMSE prueba: 2.12

El modelo explica aproximadamente el 78% de la variabilidad de la calificación final en datos no vistos, lo cual indica un buen desempeño y capacidad de generalización.
La diferencia moderada entre los valores de R² sugiere que el modelo no presenta sobreajuste significativo.

## Conclusiones

Este trabajo demuestra la importancia de una adecuada preparación de datos y selección de características en problemas reales de ciencia de datos.

El modelo de regresión lineal múltiple desarrollado logró un desempeño sólido, confirmando que las variables académicas, especialmente las calificaciones parciales, son los principales predictores del desempeño final.

Entre las limitaciones del modelo se encuentra su naturaleza lineal y la ausencia de variables externas como motivación o contexto familiar. Como trabajo futuro, se podrían explorar modelos no lineales o técnicas de regularización para mejorar la precisión.

El desempeño obtenido confirma que la regresión lineal múltiple es una herramienta
adecuada para este problema, siempre que se acompañe de una correcta preparación
de los datos y selección de variables.

En conclusión, el modelo cumple con el objetivo planteado y valida la relevancia de una metodología estructurada y bien justificada.